In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE
from pptx.oxml.ns import nsmap
from datetime import datetime
import requests
import logging
import random
import os
import io
from dotenv import load_dotenv

load_dotenv()

os.makedirs("presentations", exist_ok=True)
os.makedirs("logs", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("logs/presentation_gen.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

if os.getenv("GEMINI_API_KEY"):
    logger.info("API Key Found")
else:
    raise ValueError("GEMINI_API_KEY not found")

2026-02-06 20:21:11,960 - INFO - API Key Found


In [15]:
USER_TOPIC = input("Enter the presentation topic: ")
NUM_SLIDES = int(input("Number of slides (5-15): ") or "8")
logger.info(f"Generating presentation on: {USER_TOPIC} with {NUM_SLIDES} slides")

SYSTEM_PROMPT = """
You are a world-class presentation designer creating STUNNING keynote-style presentations.

RULES:
1. Keep text MINIMAL - maximum 3-4 bullet points per slide
2. Each bullet should be under 5 words (impactful phrases only)
3. Titles should be 2-4 words
4. Use power words, action verbs, emotional impact
5. Last slide: "Questions?" or "Thank You"
6. Think Apple Keynote / TED Talk style
"""

2026-02-06 20:21:29,222 - INFO - Generating presentation on: quantum computation with 10 slides


In [16]:
class Slide(BaseModel):
    title: str = Field(description="2-4 impactful words")
    bullets: List[str] = Field(description="2-4 bullets, each under 5 words")
    speaker_notes: str = Field(description="1-2 sentences for speaker")

class PresentationContent(BaseModel):
    title: str = Field(description="Main title, 2-5 words")
    subtitle: str = Field(description="Catchy tagline, under 6 words")
    slides: List[Slide] = Field(description="Content slides")
    author: str = Field(description="Author name, default Sougata")

In [17]:
def fetch_image(width: int = 1920, height: int = 1080) -> bytes:
    seed = random.randint(1, 1000)
    url = f"https://picsum.photos/seed/{seed}/{width}/{height}"
    try:
        response = requests.get(url, timeout=15, allow_redirects=True)
        if response.status_code == 200 and len(response.content) > 1000:
            logger.info(f"Fetched image: {len(response.content)} bytes")
            return response.content
    except Exception as e:
        logger.warning(f"Image fetch failed: {e}")
    return None

GRADIENT_COLORS = [
    (RGBColor(15, 15, 35), RGBColor(45, 45, 85)),
    (RGBColor(20, 30, 48), RGBColor(36, 59, 85)),
    (RGBColor(30, 30, 30), RGBColor(60, 60, 80)),
    (RGBColor(25, 25, 50), RGBColor(50, 50, 100)),
    (RGBColor(10, 20, 40), RGBColor(30, 60, 90)),
]

In [18]:
def add_gradient_bg(slide, prs, color1, color2):
    shape = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, 0, 0, prs.slide_width, prs.slide_height)
    shape.line.fill.background()
    fill = shape.fill
    fill.gradient()
    fill.gradient_angle = 45
    fill.gradient_stops[0].color.rgb = color1
    fill.gradient_stops[1].color.rgb = color2
    spTree = slide.shapes._spTree
    spTree.insert(2, shape._element)

def add_image_with_overlay(slide, prs, img_data, left=0, top=0, width=None, height=None):
    width = width or prs.slide_width
    height = height or prs.slide_height
    pic = slide.shapes.add_picture(io.BytesIO(img_data), left, top, width, height)
    
    overlay = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, left, top, width, height)
    overlay.fill.solid()
    overlay.fill.fore_color.rgb = RGBColor(0, 0, 0)
    overlay.fill.fore_color.brightness = -0.5
    overlay.line.fill.background()
    
    from lxml import etree
    spPr = overlay._element.find('.//p:spPr', {'p': 'http://schemas.openxmlformats.org/presentationml/2006/main'})
    if spPr is None:
        spPr = overlay._element.find('.//{http://schemas.openxmlformats.org/drawingml/2006/main}spPr')

def create_pptx(content: PresentationContent, topic: str) -> str:
    prs = Presentation()
    prs.slide_width = Inches(13.333)
    prs.slide_height = Inches(7.5)
    
    accent = RGBColor(0, 180, 255)
    
    # ===== TITLE SLIDE =====
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    img = fetch_image(1920, 1080)
    if img:
        pic = slide.shapes.add_picture(io.BytesIO(img), 0, 0, prs.slide_width, prs.slide_height)
        spTree = slide.shapes._spTree
        spTree.insert(2, pic._element)
        
        overlay = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, 0, 0, prs.slide_width, prs.slide_height)
        overlay.fill.solid()
        overlay.fill.fore_color.rgb = RGBColor(0, 0, 0)
        overlay.line.fill.background()
    else:
        colors = random.choice(GRADIENT_COLORS)
        add_gradient_bg(slide, prs, colors[0], colors[1])
    
    line = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, Inches(1), Inches(3.2), Inches(2), Pt(4))
    line.fill.solid()
    line.fill.fore_color.rgb = accent
    line.line.fill.background()
    
    title_box = slide.shapes.add_textbox(Inches(1), Inches(2.2), Inches(11), Inches(1))
    tf = title_box.text_frame
    p = tf.paragraphs[0]
    p.text = content.title.upper()
    p.font.size = Pt(64)
    p.font.bold = True
    p.font.color.rgb = RGBColor(255, 255, 255)
    
    sub = slide.shapes.add_textbox(Inches(1), Inches(3.5), Inches(11), Inches(0.8))
    tf = sub.text_frame
    p = tf.paragraphs[0]
    p.text = content.subtitle
    p.font.size = Pt(28)
    p.font.color.rgb = accent
    
    author = slide.shapes.add_textbox(Inches(1), Inches(6), Inches(11), Inches(0.5))
    tf = author.text_frame
    p = tf.paragraphs[0]
    p.text = f"Presented by {content.author}"
    p.font.size = Pt(18)
    p.font.color.rgb = RGBColor(200, 200, 200)
    
    # ===== CONTENT SLIDES =====
    for idx, slide_content in enumerate(content.slides):
        slide = prs.slides.add_slide(prs.slide_layouts[6])
        
        # Alternate: image right or gradient bg
        if idx % 2 == 0:
            img = fetch_image(800, 1080)
            if img:
                pic = slide.shapes.add_picture(
                    io.BytesIO(img), Inches(7), 0, Inches(6.333), prs.slide_height
                )
            
            left_bg = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, 0, 0, Inches(7.5), prs.slide_height)
            left_bg.fill.solid()
            left_bg.fill.fore_color.rgb = RGBColor(18, 18, 28)
            left_bg.line.fill.background()
            spTree = slide.shapes._spTree
            spTree.insert(2, left_bg._element)
        else:
            colors = GRADIENT_COLORS[idx % len(GRADIENT_COLORS)]
            add_gradient_bg(slide, prs, colors[0], colors[1])
        
        # Accent line
        line = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, Inches(0.5), Inches(0.5), Pt(5), Inches(1.2))
        line.fill.solid()
        line.fill.fore_color.rgb = accent
        line.line.fill.background()
        
        # Title
        title_box = slide.shapes.add_textbox(Inches(0.8), Inches(0.5), Inches(6), Inches(1.2))
        tf = title_box.text_frame
        p = tf.paragraphs[0]
        p.text = slide_content.title
        p.font.size = Pt(44)
        p.font.bold = True
        p.font.color.rgb = RGBColor(255, 255, 255)
        
        # Bullets
        bullet_box = slide.shapes.add_textbox(Inches(0.8), Inches(2.2), Inches(6), Inches(4.5))
        tf = bullet_box.text_frame
        tf.word_wrap = True
        
        for i, bullet in enumerate(slide_content.bullets):
            p = tf.paragraphs[0] if i == 0 else tf.add_paragraph()
            p.text = f"▸ {bullet}"
            p.font.size = Pt(26)
            p.font.color.rgb = RGBColor(230, 230, 230)
            p.space_before = Pt(20)
            p.space_after = Pt(8)
        
        # Slide number
        num_box = slide.shapes.add_textbox(Inches(12.5), Inches(6.8), Inches(0.5), Inches(0.4))
        tf = num_box.text_frame
        p = tf.paragraphs[0]
        p.text = f"{idx + 2}"
        p.font.size = Pt(14)
        p.font.color.rgb = RGBColor(120, 120, 120)
        
        notes = slide.notes_slide
        notes.notes_text_frame.text = slide_content.speaker_notes
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe = "".join(c if c.isalnum() or c in " _-" else "_" for c in topic[:25])
    filename = f"presentations/{timestamp}_{safe}.pptx"
    prs.save(filename)
    logger.info(f"Saved: {filename}")
    return filename

In [19]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
structured_model = model.with_structured_output(PresentationContent)

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("user", "Create a stunning presentation on: {topic}. Generate {num_slides} slides. Apple Keynote style.")
])

chain = prompt | structured_model

logger.info("Generating content...")
result = chain.invoke({"topic": USER_TOPIC, "num_slides": NUM_SLIDES})
logger.info("Content ready!")

print("\n🎨 Creating stunning presentation with images...")
filename = create_pptx(result, USER_TOPIC)

print("\n" + "="*60)
print("🎯 PRESENTATION CREATED!")
print("="*60)
print(f"📌 {result.title}")
print(f"✨ {result.subtitle}")
print(f"📊 {len(result.slides) + 1} slides")
print(f"📁 {filename}")
print("="*60)

for i, s in enumerate(result.slides, 2):
    print(f"{i}. {s.title}")

2026-02-06 20:21:45,293 - INFO - Generating content...
2026-02-06 20:21:45,296 - INFO - AFC is enabled with max remote calls: 10.
2026-02-06 20:21:54,664 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2026-02-06 20:21:54,668 - INFO - Content ready!



🎨 Creating stunning presentation with images...


2026-02-06 20:21:56,775 - INFO - Fetched image: 284011 bytes
2026-02-06 20:22:00,156 - INFO - Fetched image: 106091 bytes
2026-02-06 20:22:04,393 - INFO - Fetched image: 128405 bytes
2026-02-06 20:22:19,715 - WARNING - Image fetch failed: HTTPSConnectionPool(host='picsum.photos', port=443): Read timed out. (read timeout=15)
2026-02-06 20:22:33,515 - INFO - Fetched image: 84753 bytes
2026-02-06 20:22:36,311 - INFO - Fetched image: 177492 bytes
2026-02-06 20:22:36,404 - INFO - Saved: presentations/20260206_202236_quantum computation.pptx



🎯 PRESENTATION CREATED!
📌 Quantum Unleashed
✨ Beyond Classical Limits
📊 10 slides
📁 presentations/20260206_202236_quantum computation.pptx
2. What is Quantum?
3. Qubits: New Bits
4. Entanglement: Connected
5. Interference: Probabilities
6. Algorithm Breakthroughs
7. Transformative Applications
8. Hurdles & Horizon
9. Future Unlocked
10. Questions?
